We repeat the test, with Aspen-9 backend.

In [5]:
import numpy as np

from utils import bf
from QVC import PyquilVariationalClassifier
from pyquil_circuits import PauliFeatureMap, VariationalCircuit

In [6]:
# Define quantum feature map
qfm = PauliFeatureMap(num_qubits=3, rep=2)
vc = VariationalCircuit(num_qubits=3, rep=2)
qvc = PyquilVariationalClassifier(qfm, vc, bool_ftn=bf, use_bias=False)

In [7]:
# Load Data
train_data = np.load('./npy_files/TrainData_zzpfmc12_pyquil.npy')
test_data = np.load('./npy_files/TestData_zzpfmc12_pyquil.npy')
train_labels = np.load('./npy_files/TrainLabels_zzpfmc12_pyquil.npy')
test_labels = np.load('./npy_files/TestLabels_zzpfmc12_pyquil.npy')
opt_param = np.load('./npy_files/Optimal_param_zzpfmc12_pyquil.npy')

qvc.optimal_params = opt_param

In [8]:
test_acc, _, test_correct = qvc.test(test_data, (-1)**test_labels, verbose=True, backend='Aspen-9')
print(f"Test accuracy     = {test_acc}")

WARNING - Timeout on client tcp://bf04.qpu.production.qcs.rigetti.com:50053, method name execute_qpu_request, class info: <rpcq._client.Client object at 0x7f4e003106a0>, retrying in 0 seconds...


TimeoutError: Timeout on client tcp://bf04.qpu.production.qcs.rigetti.com:50053, method name execute_qpu_request, class info: <rpcq._client.Client object at 0x7f4e00310b80>

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
colors = ['tab:blue', 'tab:red', 'tab:gray']
correct_dp_c0 = list()
correct_dp_c1 = list()
false_dp = list()
for i, (dp, l) in enumerate(zip(test_data[:len(test_correct)], test_correct)):
    if l :
        if test_labels[i] == 0:
            correct_dp_c0.append(dp)
        elif test_labels[i] == 1:
            correct_dp_c1.append(dp)
        else:
            print("something went wrong")
            raise ValueError
    else:
        false_dp.append(dp)
correct_dp_c0 = np.array([np.array(d) for d in correct_dp_c0]).swapaxes(0,1)
correct_dp_c1 = np.array([np.array(d) for d in correct_dp_c1]).swapaxes(0,1)
false_dp = np.array([np.array(d) for d in false_dp]).swapaxes(0,1)
ax.scatter(correct_dp_c0[0], correct_dp_c0[1], correct_dp_c0[2],
           c = colors[0],
           label='correct_cls0')
ax.scatter(correct_dp_c1[0], correct_dp_c1[1], correct_dp_c1[2],
           c = colors[1],
           label='correct_cls1')
ax.scatter(false_dp[0], false_dp[1], false_dp[2],
           c = colors[2],
           label='false')
ax.set_xlabel('pc1')
ax.set_ylabel('pc2')
ax.set_zlabel('pc3')
fig.legend(bbox_to_anchor=(1,1))

plt.savefig('./figures/zzzpfm_c12_class_test_qpu.png')